In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

#import torchvision
#import torchvision.transforms as transforms

In [2]:
# If you need to recreate the datase, uncomment the next line
#%run create_dataset.ipynb

In [3]:
ds = pd.read_excel('dataset.xlsx', parse_dates=[0], index_col=0)

In [4]:
# The last two columns in data set are the real future data to be predicted
X_ = ds.iloc[:,:-2].values
# The last by one column is a value to be predicted
y_ = ds.iloc[:,-2].values * 100 # Prediction scaler

In [5]:
#y[y == -1] = 0

In [6]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2)

In [7]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Create dataset and dataloader
#ds_train = TensorDataset(X, y)
#train_loader = DataLoader(ds_train, batch_size=10000, shuffle=False)

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
#device = torch.device('cpu')

In [11]:
# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

In [12]:
#Create dataset and dataloader
#ds_train = TensorDataset(X_train, y_train)
#train_loader = DataLoader(ds_train, batch_size=10000, shuffle=True)

In [13]:
#ds_test = TensorDataset(X_test, y_test)
#test_loader = DataLoader(ds_test, batch_size=10000, shuffle=False)

In [14]:
# Create NN

In [15]:
class Net(nn.Module):
    def __init__(self, name=None):
        super(Net, self).__init__()
        if name:
            self.name = name
        self.fc1 = nn.Linear(30, 28)
        self.fc2 = nn.Linear(28, 27)
        self.fc3 = nn.Linear(27, 22)
        self.fc4 = nn.Linear(22, 19)
        self.fc5 = nn.Linear(19, 1)
        self.sigmoid = nn.Sigmoid()
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        x = self.fc5(x)
        return x

In [16]:
def train_model(model, X, y, criterion, optimizer, num_epochs=25):
    start = time.time()
    best_loss = float('inf')
    best_model = model
    
    model.train()
    for epoch in range(num_epochs):

        outputs = model(X)
        loss = criterion(outputs, y)
        optimizer.zero_grad()
        loss.backward()

        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = model

        optimizer.step()

        if (epoch+1) % int(num_epochs / 20) == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.8f}')

    print(f'Finished Training, best model with loss: {loss.item():.8f}')
    model = best_model
    model.eval()
    end = time.time()
    print('training time ', end - start)

In [17]:
net = Net('NN 28-27-22-19').to(device)
#net = torch.load('model_after_run_nn.pth')
criterion = nn.MSELoss()

NN 28-27-22-19: total params: 2724


In [18]:
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-8)
train_model(net, X_train, y_train, criterion, optimizer, num_epochs=200000)

Epoch [10000/200000], Loss: 0.31373087
Epoch [20000/200000], Loss: 0.22174183
Epoch [30000/200000], Loss: 0.17339225
Epoch [40000/200000], Loss: 0.14635111
Epoch [50000/200000], Loss: 0.12811467
Epoch [60000/200000], Loss: 0.11615847
Epoch [70000/200000], Loss: 0.10714879
Epoch [80000/200000], Loss: 0.09962326
Epoch [90000/200000], Loss: 0.09197962
Epoch [100000/200000], Loss: 0.08643211
Epoch [110000/200000], Loss: 0.08132211
Epoch [120000/200000], Loss: 0.07647118
Epoch [130000/200000], Loss: 0.07499529
Epoch [140000/200000], Loss: 0.06889779
Epoch [150000/200000], Loss: 0.06591066
Epoch [160000/200000], Loss: 0.06397809
Epoch [170000/200000], Loss: 0.06043433
Epoch [180000/200000], Loss: 0.05814252
Epoch [190000/200000], Loss: 0.05612484
Epoch [200000/200000], Loss: 0.05452114
Finished Training, best model with loss: 0.05452114
training time  206.20473766326904


In [19]:
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=1e-8)
train_model(net, X_train, y_train, criterion, optimizer, num_epochs=1000000)

Epoch [50000/1000000], Loss: 0.04907920
Epoch [100000/1000000], Loss: 0.04556113
Epoch [150000/1000000], Loss: 0.04276644
Epoch [200000/1000000], Loss: 0.04049289
Epoch [250000/1000000], Loss: 0.03859017
Epoch [300000/1000000], Loss: 0.03666690
Epoch [350000/1000000], Loss: 0.03500682
Epoch [400000/1000000], Loss: 0.03352980
Epoch [450000/1000000], Loss: 0.03259426
Epoch [500000/1000000], Loss: 0.03199252
Epoch [550000/1000000], Loss: 0.03138289
Epoch [600000/1000000], Loss: 0.03087592
Epoch [650000/1000000], Loss: 0.03047173
Epoch [700000/1000000], Loss: 0.03012018
Epoch [750000/1000000], Loss: 0.02972123
Epoch [800000/1000000], Loss: 0.02938945
Epoch [850000/1000000], Loss: 0.02904428
Epoch [900000/1000000], Loss: 0.02868572
Epoch [950000/1000000], Loss: 0.02825257
Epoch [1000000/1000000], Loss: 0.02798982
Finished Training, best model with loss: 0.02798982
training time  1026.2480957508087


In [20]:
# Predict the values
net.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_pred = net(X_train).cpu()

# Calculate accuracy
accuracy = accuracy_score(np.sign(y_train.cpu()), np.sign(y_pred))
print(f'Accuracy for the training set: {accuracy:.4f}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(np.sign(y_train.cpu()), np.sign(y_pred))
print('Confusion Matrix:')
print(conf_matrix)

Accuracy for the training set: 0.9842
Confusion Matrix:
[[2276   58]
 [  45 4150]]


In [21]:
# Predict the values
net.eval()  # Set the model to evaluation mode
with torch.no_grad():
    y_pred = net(X_test).cpu()

# Calculate accuracy
accuracy = accuracy_score(np.sign(y_test.cpu()), np.sign(y_pred))
print(f'Accuracy for the testing set: {accuracy:.4f}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(np.sign(y_test.cpu()), np.sign(y_pred))
print('Confusion Matrix:')
print(conf_matrix)

Accuracy for the testing set: 0.9510
Confusion Matrix:
[[ 536   49]
 [  31 1017]]


In [22]:
torch.save(net, 'model_after_run_nn.pth')

# Conclusion
The fact that the model guesses the direction of the market movement in 95 cases out of a hundred on the test data set is phenomenal. The good news is that on the training data set this value is 98%, which indicates that the model was not overfitted and it has a reasonable degree of generalization.
It would be interesting to see the potential ROI of a trading machine if it is buid on such predictions.

### Add prediction to the SP500 dataset

In [23]:
sp500 = pd.read_excel('sp500.xlsx', parse_dates=[0], index_col=0)

In [34]:
X_scaled = scaler.transform(X_)
X = torch.tensor(X_scaled, dtype=torch.float32).to(device)

In [68]:
sp500['Prediction'] = net(X).cpu().detach().numpy()/100

In [70]:
sp500.to_excel('sp500.xlsx')